In [1]:
from __future__ import print_function

from IPython import display

import tensorflow as tf
import os

#Dataset params
MODE = 'folder' #or 'file'
DATASET_PATH = '../Data Management/dataset/'

#Image params
N_CLASSES = 2 #total number of classes
IMG_HEIGHT = 64 #the image height to be resized to
IMG_WIDTH = 64 #the image width to be resized to
CHANNELS = 3 #the 3 color channels, change to 1 if grayscale

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Reading the dataset
#2 modes: 'file' or 'folder'
def read_images(dataset_path, mode, batch_size):
    imagepaths, labels = list(), list()
    if mode == 'file':
        #Read dataset file
        data = open(dataset_path, 'r').read().splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))
    
    elif mode == 'folder':
        #An ID will be affected to each sub-folders by alphabetical order
        label = 0
        #List the directory
        try: #Python 2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception: #Python 3
            classes = sorted(os.walk(dataset_path).__next__()[1])
            
        print('Classes: ', classes)
        
        #List each sub-directory (the classes)
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try: #Python 2
                walk = os.walk(c_dir).next()
            except Exception: #Python 3
                walk = os.walk(c_dir).__next__()
            
            #Add each image to the training set
            for sample in walk[2]:
                #Only keeps jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg') or sample.endswith('.png'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception('Unknown mode.')
        
    print(imagepaths)
    print(' ')
    print(labels)
    
    #Convert to tensor
    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    
    #Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer(tensor_list=[imagepaths, labels], shuffle=True)
    
    #Read images from disk
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    
    #Resize images to a common size
    image = tf.image.resize_images(image, size=[IMG_HEIGHT, IMG_WIDTH])

    
    #Normalize
    image = image * 1.0/127.5 - 1.0

    #Create batches
    x, y = tf.train.batch([image, label], 
                          batch_size=batch_size, 
                          capacity=batch_size*8, 
                          num_threads=4)
    print(x)
    print(y)
    
    return x, y

In [3]:
#Classic CNN
#Params
learning_rate = 0.001
num_steps = 1000
batch_size = 4
display_step = 100

#Network Param
dropout = 0.75 #Probability to keep units

#Build the data input
X, Y = read_images(DATASET_PATH, MODE, batch_size)

Classes:  ['bubble', 'ice']
['../Data Management/dataset/bubble\\image1.jpg', '../Data Management/dataset/bubble\\image2.jpg', '../Data Management/dataset/bubble\\image3.jpg', '../Data Management/dataset/bubble\\image4.jpg', '../Data Management/dataset/ice\\image1.jpg', '../Data Management/dataset/ice\\image2.jpg', '../Data Management/dataset/ice\\image3.jpg', '../Data Management/dataset/ice\\image4.jpg']
 
[0, 0, 0, 0, 1, 1, 1, 1]
Tensor("batch:0", shape=(4, 64, 64, 3), dtype=float32)
Tensor("batch:1", shape=(4,), dtype=int32)


In [4]:
X

<tf.Tensor 'batch:0' shape=(4, 64, 64, 3) dtype=float32>

In [5]:
Y

<tf.Tensor 'batch:1' shape=(4,) dtype=int32>

In [6]:
#Create CNN model
def conv_net(x, n_classes, dropout, reuse, is_training):
    #Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        #Convolution layer 1 with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=5, activation=tf.nn.relu)
        #MaxPooling(down-sampling) with strides 2 and kernel size 2
        conv1 = tf.layers.max_pooling2d(inputs=conv1, strides=2, pool_size=2)
        
        #Convolution layer 2 with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=3, activation=tf.nn.relu)
        #MaxPooling with strides 2 and kernel size 2
        conv2 = tf.layers.max_pooling2d(inputs=conv2, strides=2, pool_size=2)
        
        #Flatten the data to a 1-D vector for fully connected
        fc1 = tf.contrib.layers.flatten(inputs=conv2)
        
        #Fully connected layer
        fc1 = tf.layers.dense(inputs=fc1, units=1024)
        
        #Apply dropout(if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(inputs=fc1, rate=dropout, training=is_training)
        
        #Output layer
        out = tf.layers.dense(inputs=fc1, units=n_classes)
        
        #Because softmax_cross_entropy_with_logits already apply softmax,
        #we only apply softmax to testing network
#         out = tf.nn.softmax(out) if not is_training else out
        
    return out

In [7]:
#Because dropout have different behavior at training and prediction time, we
#need to create 2 distinct computation graphs that share the same weights

#Tf graph input
x = tf.placeholder(dtype=tf.float32, shape=[None, IMG_HEIGHT, IMG_WIDTH, CHANNELS])
y = tf.placeholder(dtype=tf.float32, shape=[None, N_CLASSES])

#Create a graph for training
logits_train = conv_net(x=X, n_classes=N_CLASSES, dropout=dropout, reuse=False, is_training=True)
prediction = tf.nn.softmax(logits_train)
#Create another graph for testing that reuse the same weights
logits_test = conv_net(x=X, n_classes=N_CLASSES, dropout=dropout, reuse=True, is_training=False)

#Define loss and optimizer(with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluate model(with test logits for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(Y, dtype=tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variables
init = tf.global_variables_initializer()

#Saver object
saver = tf.train.Saver()

In [8]:
#Start training
with tf.Session() as sess:
    sess.run(init)
    
    #Start the data queue
    tf.train.start_queue_runners()
    
    #Train cycle
    for step in range(1, num_steps+1):
#         batch_x = tf.train.batch(tensors=X, batch_size=batch_size)
#         batch_y = tf.train.batch(tensors=Y, batch_size=batch_size)
        if step % display_step == 0:
            #Run the optimization and calculate batch loss and accuracy
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            
            print('Step %d, Minibatch Loss= %.4f, Training Accuracy= %.3f' % (step, loss, acc))
        else:
            #Only run the optimization op
            sess.run(train_op)
            
    print('Training Completed!!!')
    
    #Save model
    saver.save(sess, 'my_tf_model')

Step 100, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 200, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 300, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 400, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 500, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 600, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 700, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 800, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 900, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 1000, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Training Completed!!!
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: input_producer/input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer/input_producer, input_producer/input_producer/RandomShuffle)]]
ERROR:tensorflow:Exception in QueueRunner: Enqueue 

Exception in thread QueueRunnerThread-input_producer/input_producer-input_producer/input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "C:\Users\StrikeWade\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\StrikeWade\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\StrikeWade\Anaconda3\lib\site-packages\tensorflow\python\training\queue_runner_impl.py", line 250, in _run
    enqueue_callable()
  File "C:\Users\StrikeWade\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1251, in _single_operation_run
    self._session, None, {}, [], target_list, status, None)
  File "C:\Users\StrikeWade\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 473, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[Node: input_producer/in

ValueError: Parent directory of my_tf_model doesn't exist, can't save.